# API APN NLT

## Libs

In [ ]:
import pandas as pd
import numpy as np
import requests
import json

## API

In [ ]:
url = "https://gsm-clients.nlt-iot.com"

### Get credentials

In [ ]:
def getToken():
    with open('auth.json', 'r') as f:
        auth = json.load(f)
    
    payload = json.dumps({
      "email": auth['email'],
      "password": auth['password']
    })
    headers = {
      'Content-Type': 'application/json'
    }

    response = requests.request("POST", url + "/token", headers=headers, data=payload)
    return response.json()['access_token']

In [ ]:
token = getToken()

### Get status

In [ ]:
#msisdn = "5511..."

In [ ]:
def getStatus (msisdn, token):
    payload={}
    headers = {
      'Authorization': 'Bearer ' + token
    }

    response = requests.request("GET", url + "/v1/lines/status/" + msisdn, headers=headers, data=payload)
    return response.json()

In [ ]:
#status = getStatus(msisdn, token)

### Check APN 

In [ ]:
defaultAPN = "apn.xpto"

In [ ]:
def checkForAPN(apnToBeFound, status):
    isAPN = False
    apns = status['apns']
    for apn in apns:
        if apn.find(apnToBeFound) != -1:
            isAPN = True

    return isAPN

In [ ]:
#checkForAPN(defaultAPN, status)

### Add APN 

In [ ]:
def addAPN(apn_name, msisdn, token):
    payload = json.dumps({
      "msisdn": msisdn,
      "apn": apn_name
    })
    headers = {
      'Authorization': 'Bearer ' + token,
      'Content-Type': 'application/json'
    }
    response = requests.request("POST", url + "/v1/lines/provisioning-apn", headers=headers, data=payload)
    
    return response.json()

In [ ]:
resp = addAPN(defaultAPN, msisdn, token)

## Mass aprovisioning

In [ ]:
df = pd.read_excel("data/lista_msisdn.xlsx")
df.shape

In [ ]:
lotes = [1234, 5678]
df_lotes = df[df["Nº LOTE"].isin(lotes)]

df_lotes.shape

In [ ]:
token = getToken()

In [ ]:
has_apn = []
has_not_apn = []

for index, row in df_lotes.iterrows():
    msisdn = str(row["NÚMERO"])
    status = getStatus(msisdn, token)
    if checkForAPN(defaultAPN, status):
        has_apn.append(msisdn)
        print("APN is already setup in ", msisdn)
    else:
        has_not_apn.append(msisdn)
        resp = addAPN(defaultAPN, msisdn, token)
        print(resp)

In [ ]:
len(has_not_apn)

In [ ]:
len(has_apn)